This is me: overwhelmed by ideas such that they become difficult to work on.<br>
A partner, to hold finese all those rough edges I missed because you were tired <br>

In [1]:
from langchain.document_loaders import TextLoader

In [2]:
articles = ["making-effect-career-change.txt",
                     "python-on-the-go.txt",
                     "safety-net-ep-0.txt",
                     "safety-net-ep-1.txt",
                     "safety-net-ep-2.txt"]

In [3]:
article_docs = []
for article in articles:
    loader = TextLoader(article)
    doc = loader.load()
    article_docs += doc

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectorstore = FAISS.from_documents(
    documents=article_docs,
    embedding=embedding)

In [142]:
vectorstore.save_local("articles")

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

retriever = vectorstore.as_retriever()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

In [13]:
import numpy as np
np.mean([len(i.page_content.split()) for i in article_docs])

np.float64(502.0)

In [136]:
from langchain.prompts import PromptTemplate

template = """
You are supportive friend who enjoys my articles and you really like my style of writing.
These are all the articles you have read and liked: {context}.
I am working on some new material and I need your help to bring out the best in my writing.
You are well aware of my writing style, my humor and the kind of English I use.
I will provide you with a draft of what I am working on, and you should help me in reviewing and suggesting
changes I should make and mention things that you appreciate from the draft: {question}.
Give a complete review with the following sections: The Best, The Good, The Meh, The Bad and The Ugly. Be thorough in your review.
Speak in a friendly conversational manner. When listing items, use numbers instead of *
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [138]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
            {"context": retriever, "question": RunnablePassthrough()} |
            prompt |
            llm |
            StrOutputParser()
    )

In [139]:
fresh_article = """
Looking for the friendliest people in the world, try scammers..

Scammers will treat you like royalty and make you feel like nothing can stop you..well, that is before they steal your accounts or empty your bank balance.

As a data scientist, I came across this dataset of spam emails. Let me tell you, that was one of the most entertaining dataset I had ever come across. 

There were pretty interesting scammer emails, and yeah, the Nigerian Prince even said "Hello". Reading through them (not because I was enjoying reading them but trying to find a pattern) I realized the building blocks of the scam are the same even as today.

Although you might feel a sense of self-worthiness when you detect what's being done to you is a scam, sometimes scams can catch you off-guard. And the interesting thing is scams can play with your curiosity. That little piece of you that wants to know if the scammer was telling the truth, that curious bit which wonders if you could have made it if you had followed the scammer, that sense could be your doom.

Some scams I have come across personally are "The Lucky Winner", and they usually come up with a relevant context that can drive the unwary to be scammed. Relevant context may involve company anniversaries and even emails that show resemblance to the one of the acclaimed company.

And some scams are just there to waste your time and drive traffic on some sites. Although, I admire the "Share to 10" nonsense, the person who came up with that is a genius! And also hand down to the person who came up with the "Download these 3 Apps on Playstore to get this impossible thing to find" technique. But genius as it may be, in a community like Zim/Africa. I can imagine a young, misguided person pouring their hard-earned data into a bucket with holes. And I know the feeling of someone coming up behind you while you are surfing them sites and a questionable ad pops up and creates the "I can explain" situation.

I still find it more sad than funny, to see people in a tech WhatsApp group, those people you would expect to be gurus of tech (and of course those who see themselves as more "evolved" than the less tech-literate) share scam posts in the group.  Believe me, I'm not saying I can smell all scams a mile away but there is this level of intuition that I think everyone has that tells them that 

there's something wrong with this. 

One of the problem is also the trust we put in people who shared the suspicious content with us. Like "it was sent to me by X" so it's legit. Scams are all around

And let's not forget scholarships, everyone want an easy scholarship to study in Canada with everything paid for and a big figure stated stipend. Please note that I'm not saying every opportunity you come across is a scan but this acts in helping you evaluate individually.
"""

In [140]:
response = chain.invoke(fresh_article)

In [141]:
print(response)

Hey! This is a great start! I always look forward to reading your articles. You have a way of making even serious topics like scams engaging and relatable with your personal anecdotes and humor. Your writing style is unique and I love how you weave in your own experiences and observations. Keep it up!

Here's my breakdown:

**The Best:**

1.  **Your Voice:** Your personality shines through! The way you phrase things, like "I can imagine a young, misguided person pouring their hard-earned data into a bucket with holes," is so uniquely you. It's what makes your writing stand out.
2.  **Relatability:** The "I can explain" situation with questionable ads is hilarious and so relatable. Everyone has been there! You have a knack for tapping into common experiences.
3.  **Humor:** The opening line about scammers treating you like royalty is gold. It grabs attention immediately and sets the tone for the article.

**The Good:**

1.  **Intriguing Hook:** Starting with the spam email dataset as a 